In [1]:
import requests
import json
#import logging
#import http.client
import pandas as pd
import xlwings as xw
from time import sleep
from datetime import datetime,time,timedelta
import os
#import property as p
import numpy as np
import matplotlib.pyplot as plt
#from scipy.interpolate import make_interp_spline, BSplin
import seaborn



pd.set_option("display.width",1500)
pd.set_option("display.max_columns",75)
pd.set_option("display.max_rows",150)
%config Completer.use_jedi = False

In [2]:
def readJson(filename):
    with open(filename,"r") as files:
        data = json.load(files)
    #print(data)
    return data

In [50]:
df_list = readJson("oi_data_records_310321.json")
df = pd.DataFrame()
for item in df_list:
    df = pd.concat([df,pd.DataFrame(item)])
#print(df)



In [4]:
df.shape

(11680, 21)

In [5]:
df1 = df[df['expiryDate'] == '01-Apr-2021']

In [6]:
df2 = df1[(df1['lastPrice']  !=0)   & (df1['impliedVolatility']  !=0) & (df1['Time']== df1['Time'].max())]

print(df2['type'].unique())

print(int(df2['underlyingValue'].unique()[0]))


['CE' 'PE']
33320


In [7]:
##df2 = df1.groupby(['Time','underlyingValue'])[['change', 'changeinOpenInterest', 'expiryDate']].size().reset_index().rename(columns = {0:'count'})
df2.columns

Index(['Time', 'askPrice', 'askQty', 'bidQty', 'bidprice', 'change', 'changeinOpenInterest', 'expiryDate', 'identifier', 'impliedVolatility', 'lastPrice', 'openInterest', 'pChange', 'pchangeinOpenInterest', 'strikePrice', 'totalBuyQuantity', 'totalSellQuantity', 'totalTradedVolume', 'type', 'underlying', 'underlyingValue'], dtype='object')

In [68]:
def findStrikerangeDataframe(df, underlyingvalue , rangeval):
    start = (underlyingvalue-underlyingvalue%500)-rangeval
    end = (underlyingvalue-underlyingvalue%500) +rangeval
    strikerang = [*range(start,end,500)]
    atm =underlyingvalue-underlyingvalue%100 
    strikerang.append(atm)
    print(strikerang)
    df1 = df[df['strikePrice'].isin (strikerang)]
    range_df = df1.drop(['Time','askPrice', 'askQty', 'bidQty', 'bidprice','totalBuyQuantity', 'totalSellQuantity','underlying','totalTradedVolume','identifier','underlyingValue','expiryDate'], axis=1)
    return range_df,atm


range_df, atm = findStrikerangeDataframe(df2,int(df2['underlyingValue'].unique()[0]),2000)
print(atm)

newdf =(range_df.groupby('type')['changeinOpenInterest'].nlargest(3).reset_index().rename(columns = {'level_1':'n'}) )
print((range_df[range_df.index.isin(newdf['n'])]))
#print(range_df[range_df['changeinOpenInterest']== 28552])
#print(range_df.loc[[62]])

#print(int(df2['underlyingValue'].unique()[0]))
#int(df2['underlyingValue'].unique()[0])%500


[31000, 31500, 32000, 32500, 33000, 33500, 34000, 34500, 33300]
33300
     change  changeinOpenInterest  impliedVolatility  lastPrice  openInterest    pChange  pchangeinOpenInterest  strikePrice type
57  -486.45                  7615              44.03     533.00         15970 -47.716906              91.143028        33000   CE
62  -403.30                 28552              40.01     223.80         43681 -64.311912             188.723643        33500   CE
67  -263.95                 12285              37.98      61.70         55912 -81.053278              28.159167        34000   CE
183  -10.35                  3352              42.70      45.35         26808 -18.581688              14.290587        32500   PE
188   12.70                 10647              39.50     127.55         38482  11.057902              38.250404        33000   PE
191   49.55                  7912              37.48     224.95         15810  28.249715             100.177260        33300   PE
